GOJEK Directors have asked analysts to look at the data to understand what has happened during Q1 2016 and what they should do to maximize the revenue for Q2 2016. Given the data in this table :
1. What are the main problems that we need to focus on? State your findings clearly.
2. Present your findings from question 1. The goal is to persuade management to make a decision based on your suggestions, so it is important that the findings are intuitively depicted

In [1]:
import pandas as pd
#pd.set_option('display.float_format', lambda x: '%.10f' % x)
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_excel('Analytics + Viz test - DeS_DA - Raw data - 11_05_2021.xlsx')
df['Total CBV'] = df['Total CBV'].astype(str)
df['Total CBV'] = df['Total CBV'].str.replace('.', '')
df['Total CBV'] = df['Total CBV'].str.replace('-', '0')
df['Total CBV'] = df['Total CBV'].astype('int64')
df.head()

,Date,Month,# of Orders,Status of Order,Total CBV,Service
0,2016-01-01 00:00:00,1,180,Cancelled,76076000,GO-BOX
1,2016-01-01 00:00:00,1,10,Cancelled,1960140,GO-CLEAN
2,2016-01-01 00:00:00,1,28380,Cancelled,489898500,GO-FOOD
3,2016-01-01 00:00:00,1,0,Cancelled,728,GO-GLAM
4,2016-01-01 00:00:00,1,10,Cancelled,1365,GO-KILAT


In [3]:
print ('The data has {0} rows and {1} columns'.format(df.shape[0],df.shape[1]))

The data has 3171 rows and 6 columns


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3171 entries, 0 to 3170
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Date             3171 non-null   object
 1   Month            3171 non-null   int64 
 2   # of Orders      3171 non-null   int64 
 3   Status of Order  3171 non-null   object
 4   Total CBV        3171 non-null   int64 
 5   Service          3171 non-null   object
dtypes: int64(3), object(3)
memory usage: 148.8+ KB


### Distribution of Status of Order 

In [6]:
order_status = df[['# of Orders', 'Status of Order']].groupby('Status of Order').sum().reset_index()
order_status['Total'] = order_status['# of Orders'].sum()
order_status['# of Orders %'] = round((order_status['# of Orders'] / order_status['Total']) * 100,2)
order_status = order_status.sort_values(by='# of Orders',ascending = False).reset_index()
order_status[['Status of Order', '# of Orders', '# of Orders %']]

,Status of Order,# of Orders,# of Orders %
0,Completed,41709050,80.12
1,Cancelled,8690110,16.69
2,No Driver Found,1643480,3.16
3,Other,8380,0.02
4,Failed/Timeout,4330,0.01


### Number of Orders across different services

In [8]:
service_status = df[['# of Orders', 'Total CBV', 'Service']].groupby('Service').sum().reset_index()
service_status['Total'] = service_status['# of Orders'].sum()
service_status['CBV'] = service_status['Total CBV'].sum()
service_status['# of Orders %'] = round((service_status['# of Orders'] / service_status['Total']) * 100,2)
service_status['CBV %'] = round((service_status['Total CBV'] / service_status['CBV']) * 100,2)
service_status = service_status.sort_values(by='Total CBV',ascending = False).reset_index()
service_status = service_status[['Service', '# of Orders', 'Total CBV', '# of Orders %', 'CBV %']]
service_status

,Service,# of Orders,Total CBV,# of Orders %,CBV %
0,GO-RIDE,37164950,806848597100,71.40,67.46
1,GO-FOOD,8698770,145743172939,16.71,12.18
2,GO-SEND,3646690,133639592086,7.01,11.17
3,GO-SHOP,2035620,51526604402,3.91,4.31
4,GO-BOX,96750,40386596654,0.19,3.38
5,GO-MASSAGE,52500,9276102927,0.10,0.78
6,GO-MART,279820,3911947858,0.54,0.33
7,GO-CLEAN,17250,2579787301,0.03,0.22
8,GO-GLAM,10820,1592323005,0.02,0.13
9,GO-KILAT,19050,363772773,0.04,0.03


### Distribution of Status of Order across different services

In [9]:
status_service = df[['# of Orders', 'Status of Order', 'Service']].groupby(['Service','Status of Order']).sum()
e =  pd.pivot_table(status_service, values = '# of Orders', index = ['Service'], columns = ['Status of Order'], aggfunc = np.sum, fill_value = 0)
e['Total'] = e.sum(axis=1)
for i in e.columns:
    e[i] = (e[i] / e['Total']) * 100
    e[i] = round(e[i],2)
e = e.sort_values(by='Cancelled',ascending=False).reset_index()
e

Status of Order,Service,Cancelled,Completed,Failed/Timeout,No Driver Found,Other,Total
0,GO-TIX,69.03,17.90,13.07,0.00,0.00,100.0
1,GO-SHOP,32.26,66.40,0.00,1.33,0.01,100.0
2,GO-BOX,30.53,65.90,0.00,3.57,0.00,100.0
3,GO-MART,30.41,69.45,0.00,0.14,0.00,100.0
4,GO-FOOD,23.20,76.33,0.00,0.45,0.02,100.0
5,GO-RIDE,14.50,81.56,0.00,3.92,0.01,100.0
6,GO-MASSAGE,13.64,79.89,0.00,5.71,0.76,100.0
7,GO-SEND,13.07,83.95,0.00,2.98,0.01,100.0
8,GO-GLAM,11.46,69.32,0.00,18.85,0.37,100.0
9,GO-CLEAN,9.86,82.78,0.00,3.48,3.88,100.0


In [10]:
a = pd.merge(service_status, e, on = 'Service', how = 'inner')
a

,Service,# of Orders,Total CBV,# of Orders %,CBV %,Cancelled,Completed,Failed/Timeout,No Driver Found,Other,Total
0,GO-RIDE,37164950,806848597100,71.40,67.46,14.50,81.56,0.00,3.92,0.01,100.0
1,GO-FOOD,8698770,145743172939,16.71,12.18,23.20,76.33,0.00,0.45,0.02,100.0
2,GO-SEND,3646690,133639592086,7.01,11.17,13.07,83.95,0.00,2.98,0.01,100.0
3,GO-SHOP,2035620,51526604402,3.91,4.31,32.26,66.40,0.00,1.33,0.01,100.0
4,GO-BOX,96750,40386596654,0.19,3.38,30.53,65.90,0.00,3.57,0.00,100.0
5,GO-MASSAGE,52500,9276102927,0.10,0.78,13.64,79.89,0.00,5.71,0.76,100.0
6,GO-MART,279820,3911947858,0.54,0.33,30.41,69.45,0.00,0.14,0.00,100.0
7,GO-CLEAN,17250,2579787301,0.03,0.22,9.86,82.78,0.00,3.48,3.88,100.0
8,GO-GLAM,10820,1592323005,0.02,0.13,11.46,69.32,0.00,18.85,0.37,100.0
9,GO-KILAT,19050,363772773,0.04,0.03,5.30,88.03,0.00,6.67,0.00,100.0


In [11]:
status_service = df[['Total CBV', 'Status of Order', 'Service']].groupby(['Service','Status of Order']).sum()
e =  pd.pivot_table(status_service, values = 'Total CBV', index = ['Service'], columns = ['Status of Order'], aggfunc = np.sum, fill_value = 0)
e['Total'] = e.sum(axis=1)
for i in e.columns:
    e[i] = (e[i] / e['Total']) * 100
    e[i] = round(e[i],2)
e = e.sort_values(by='Cancelled',ascending=False).reset_index()
e

Status of Order,Service,Cancelled,Completed,Failed/Timeout,No Driver Found,Other,Total
0,GO-BOX,33.86,59.40,0.0,6.72,0.01,100.0
1,GO-SHOP,33.84,64.53,0.0,1.63,0.01,100.0
2,GO-MART,30.45,69.54,0.0,0.00,0.00,100.0
3,GO-FOOD,24.03,75.46,0.0,0.50,0.02,100.0
4,GO-RIDE,15.43,80.53,0.0,4.03,0.01,100.0
5,GO-SEND,13.58,83.47,0.0,2.95,0.01,100.0
6,GO-MASSAGE,13.42,81.19,0.0,4.64,0.75,100.0
7,GO-GLAM,10.40,69.25,0.0,20.20,0.15,100.0
8,GO-CLEAN,9.51,84.20,0.0,2.90,3.38,100.0
9,GO-KILAT,0.39,94.80,0.0,4.80,0.00,100.0
